# 필요한 라이브러리 import

In [2]:
import os
import json
import numpy as np
import pandas as pd
from scipy import sparse as spr
from itertools import chain
from collections import defaultdict,Counter
import hnswlib
import warnings
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import repeat
import random
from scipy.stats import skew,kurtosis
warnings.filterwarnings(action='ignore')

# 데이터 로드

## 노래 데이터 로드

In [3]:
with open('./data/song_meta.json',encoding='utf-8-sig') as f:
    song_dict = json.load(f)
    
song_df = pd.DataFrame.from_dict(song_dict)

## 장르 데이터 로드

In [4]:
with open('./data/genre_gn_all.json',encoding='utf-8-sig') as f:
    genre_dict = json.load(f)

## 플레이리스트 데이터 로드

In [5]:
with open('data/train.json',encoding='utf-8-sig') as f:
    train_dict = json.load(f)
    
playlist_df = pd.DataFrame.from_dict(train_dict)

# 데이터 전처리

## 태그 전처리

### tag와 tag_id 간 딕셔너리 만들기 

In [6]:
with open('./data/tag_tag_id_dict.json','r',encoding='utf-8-sig') as f:
    tag_to_id = json.load(f)

In [7]:
# id를 tag로
id_to_tag = {}
    
for k,v in tag_to_id.items() :
    id_to_tag[v] = k

### 태그 사용 빈도수 도출

In [8]:
# train dataframe tag 컬럼의 모든 tag들 (중복포함)
tags_all = playlist_df['tags'].tolist()

# 태그의 빈도수를 가진 dict, Counter 써도 됨
tags_frequency = defaultdict(int)

# 특정 tag가 나올 때마다 1더하기
for tags in tags_all:
    for tag in tags:
        tags_frequency[tag] += 1

In [9]:
# 태그 빈도의 분포를 알아보자
tag_freq = pd.DataFrame().from_dict(tags_frequency,orient="index")
tag_freq.columns=['빈도']
tag_freq.describe().T

,count,mean,std,min,25%,50%,75%,max
빈도,29160.0,16.335082,247.011075,1.0,1.0,1.0,3.0,16465.0


### 태그 사용 빈도 수의 평균 값(16)을 기준으로 태그를 절삭

In [10]:
# 빈도수로 태그를 filter

def filter_func(x):
    temp = []
    for tag in x:
        if tags_frequency[tag] >=16:
            temp.append(tag)
        else:
            pass
    return temp
            
playlist_df['tags'] = playlist_df['tags'].map(filter_func)

### 행렬을 만들기 위해 tag를 id로 만들어 새 컬럼 생성

In [11]:
# filter된 태그의 id만 남기기
playlist_df['tag_ids'] = playlist_df['tags'].map(lambda x : [tag_to_id[v] for v in x])

### 컬럼명 한글화 및 플레이리스트 당 태그의 수 컬럼 생성

In [12]:
playlist_df.columns=['태그','플리아이디','플리제목','노래들','좋아요수','변경일자','태그아이디']
playlist_df['태그수'] = playlist_df['태그'].map(len)

## 플레이리스트의 태그로 예측에 사용 될 태그와 검증에 사용 될 태그로 나누기

In [13]:
# 3:7 비율로 랜덤으로 나눌 것이기 때문에 태그수가 3 보다 커야 한다.
origin_tags = playlist_df[playlist_df['태그수']>9]['태그'].tolist()

train_tags = []
test_tags = []

# 3:7비율로 나누기
for tags in origin_tags:
    tag_3p = len(tags)//3
    train_tag = random.sample(tags, tag_3p)
    test_tag = list(set(tags)-set(train_tag))
    train_tags.append(train_tag)
    test_tags.append(test_tag)
    
cal_tag_hit_df = pd.DataFrame(columns=['예측용태그','검증용태그','als_예측결과','als_히트'])

cal_tag_hit_df['예측용태그'] = train_tags
cal_tag_hit_df['검증용태그'] = test_tags
cal_tag_hit_df.head(1)

,예측용태그,검증용태그,als_예측결과,als_히트
0,"[트로피컬하우스, 드라이브, 일렉]","[팝, 힐링, 운동, 트렌드, Pop, 2017, 기분전환]",NaN,NaN


# 세부장르 전처리

## 노래 데이터 프레임 컬럼명 한글화

In [14]:
# 노래 데이터 프레임 컬럼명 한글화
song_df.colums=['세부장르','발매일','앨범명','앨범ID','가수ID','노래명','대장르','가수명','노래ID']

## 세부 장르 이름에 대장르의 이름 넣기

In [15]:
genre_dict['GN9000'] = '기타장르'

genre_big = {}

# 모든 장르 딕셔너리를 돌면서
for k,v in genre_dict.items():
    
    # 맨 뒤 두자리가 00이면 대장류로 분류
    if k[-2:] == '00':
        
        # 맨앞 네자리를 키로 하는 대장류 딕셔너리 값 추가
        genre_big[k[:4]] = v

genre_detail_dict = {}

# 모든 딕셔너리를 돌면서
for k,v in genre_dict.items():
    
    # 맨뒤 두자리가 00이 아니면 대장류가 아닌거임!
    if k[-2:] != '00':
        
        # 그럴떈 아까만든 대장르 딕셔너리의 대장류 이름을 추가해서 이름을 수정해서 다시 넣어줌
        new_value = genre_big[k[:4]]+'_'+v
        genre_detail_dict[k] = new_value
        
genre_big_dict = {}

for k,v in genre_big.items():
    genre_big_dict[k+'00'] = v

## 세부장르ID와 CODE간 딕셔너리 생성

In [16]:
# 세부장르의 id를 세부장르 코드로
genre_detail_id_to_code = {}

# 세부장르의 코드를 세부장르의 id로
genre_detail_code_to_id = {}

for i,v in enumerate(list(genre_detail_dict.keys())):
    genre_detail_id_to_code[i] = v
    
for i,v in enumerate(list(genre_detail_dict.keys())):
    genre_detail_code_to_id[v] = i

## 노래ID와 세부장르ID간 딕셔너리 생성

In [17]:
# 노래의 ID를 KEY 세부장르의 ID리스트를 ITEM으로 하는 딕셔너리 생성 

song_genre_detail_dict = defaultdict(list)

for codes,id in zip(song_df['song_gn_dtl_gnr_basket'].tolist(),song_df['id'].tolist()):
    for code in codes:
        song_genre_detail_dict[id].append(genre_detail_code_to_id[code])

## 플레이리스트 각 노래마다 달린 세부장르의 ID를 전부 플레이리스트에 추가

In [18]:
# 플레이리스트의 모든 노래의 세부장르를 모아서 새로운 컬럼으로 생성

def fetcher(x):
    temp = []
    for song in x:
        genre_ids = song_genre_detail_dict[song]
        for id in genre_ids:
            temp.append(id)
    return temp

playlist_df['세부장르'] = playlist_df['노래들'].map(fetcher)
playlist_df.head(1)

,태그,플리아이디,플리제목,노래들,좋아요수,변경일자,태그아이디,태그수,세부장르
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,[25304],1,"[97, 96, 48, 49, 56, 67, 60, 56, 68, 48, 49, 5..."


## 태그ID와 세부장르ID들 간 딕셔너리 생성

In [19]:
# {tag_Id:genre_detail_ids}

tag_id_to_genre_detail_ids = defaultdict(list)

for ids,genres in zip(playlist_df['태그아이디'].tolist(),playlist_df['세부장르'].tolist()):
    for id in ids:
        tag_id_to_genre_detail_ids[id].extend(genres)

# 태그-세부장르 간 csr_matrix만들기

In [20]:
# 세부장르의 빈도수를 카운트해서 tag_id_to_genre_detail_id의 value를 바꿔줌

for k,v in tag_id_to_genre_detail_ids.items():
    tag_id_to_genre_detail_ids[k] = dict(Counter(v))

In [21]:
# 딕셔너리의 빈도를 min_max_scaling하고 1을 더함 -> 0이 되는것은 싫다!
for k,value_dict in tag_id_to_genre_detail_ids.items():
    max_val = np.max(list(value_dict.values()))
    min_val = np.min(list(value_dict.values()))
    for key,value in value_dict.items():
        value_dict[key] = np.round((value-min_val)/(max_val-min_val),3)

## 만든 딕셔너리를 이용해서 csr_matrix 생성

In [22]:
# 행,열,데이터 list를 생성
row = []
col = []
dat = []

for k,v in tag_id_to_genre_detail_ids.items():
    for vk,vv in v.items():
        row.append(k)
        col.append(vk)
        dat.append(vv)

In [23]:
# 29160의 태그의 가짓수, 224는 세부장르의 가짓수
A = spr.csr_matrix((dat, (row, col)), shape=(29160, 224))

# 행렬분해 수행을 위해 자료형 변경
A = A.astype(float)

# ALS로 태그 임베딩

## 태그-세부장르 행렬의 ALS 수행 

In [24]:
# als tag의 벡터들을 로드
with open('./data/als_tag_100_vectors.pickle', 'rb') as f:
    als_vectors = pickle.load(f)

## cosine유사도를 이용해서 KNN 모델 생성

In [25]:
#svd knn모델 load
with open('./data/als_100_knn_model.pickle', 'rb') as f:
    als_p = pickle.load(f)

# 태그 예측

## 예측 수행 및 수행 결과 저장

In [26]:
# svd_knn_model로 태그를 예측하여 예측된 결과의 list를 저장하는 함수

def tag_als(x):
    target_ids = [tag_to_id[t] for t in x]
    
    vectors = np.zeros((100,1))
    
    for id in target_ids:
        vectors = vectors+als_vectors[id]
    
    labels, distances = als_p.knn_query(vectors/len(target_ids), k = 10+len(x))

    ids = [label for label in labels[0]]
   
    return list(set([id_to_tag[tag] for tag in ids])-set(x))

In [27]:
# 예측수행
cal_tag_hit_df['als_예측결과'] = cal_tag_hit_df['예측용태그'].map(tag_als)

In [28]:
cal_tag_hit_df

,예측용태그,검증용태그,als_예측결과,als_히트
0,"[트로피컬하우스, 드라이브, 일렉]","[팝, 힐링, 운동, 트렌드, Pop, 2017, 기분전환]","[좋은노래, 팝송, 댄스, 세련된, 힙합, 띵곡, 취향저격, 클럽, 그루브, 소울,...",NaN
1,"[취향저격, 신나는, 여행]","[매장음악, 댄스, 여름, 드라이브, 일렉, 일렉트로니카, 기분전환]","[회상, 매장음악, 추억, 겨울, 드라이브, 운동, 감성, 사랑, 퇴근길, 설렘, ...",NaN
2,"[사랑, 감성, 우울]","[새벽, 인디, 발라드, 잔잔한, 이별, 알앤비, 기분전환]","[매장음악, 설렘, 여행, 겨울, 퇴근길, 새벽, 가을, 혼자, 봄, 기분전환]",NaN
3,"[드라이브, 라운지, 기분전환]","[세련된, 휴식, 펍, 매장, 그루브, 퇴근길, 새벽]","[매장음악, 여행, 비오는날, 여름, 감성, 매장, 카페, 새벽, 가을, 분위기, 밤]",NaN
4,"[잠, 추억, 잔잔한]","[매장음악, 일상, 추위, 매장, 카페, 자장가, 새벽]","[주말, 휴식, 비오는날, 힐링, 감성, 차분한, 카페, 배경음악, 노동요, 새벽,...",NaN
...,...,...,...,...
6896,"[찬바람, 공부, 연주곡]","[따스한, 집중, 오후, 아침, 노동요, 피아노, 겨울감성]","[일상, 잠들기전, 뉴에이지, 집중, 불면증, 밤새벽, 피곤, 카페, 피아노, 모닝...",NaN
6897,"[휴식, 기분전환, 커피]","[매장음악, 위로, 취향저격, 감성, 잔잔한, 센치, 봄]","[여행, 비오는날, 감성, 산책, 카페, 잔잔한, 새벽, 가을, 봄, 밤]",NaN
6898,"[휴식, 카페, 쌀쌀한]","[매장, 연주곡, 출근길, 아침, 노동요, 모닝콜, 피아노]","[설렘, 비오는날, 겨울, 감성, 분위기, 퇴근길, 행복, 새벽, 가을, 커피, 봄...",NaN
6899,"[팝송, 취향저격, 감성]","[신나는, 힐링, 휴식, Pop, 잔잔한, 팝송음악, 기분전환]","[회상, 띵곡, 감각적인, 드라이브, 휴가, 퇴근길, 우울, 노래, 보컬, 설렘]",NaN


## 평가 지표

In [29]:
val_tag = cal_tag_hit_df['검증용태그'].tolist()
als_tag = cal_tag_hit_df['als_예측결과'].tolist()

als_hit = []

for val,als in zip(val_tag,als_tag):
    if len(val) == len(set(val)-set(als)):
        als_hit.append(0)
    else:
        als_hit.append(1)
        
cal_tag_hit_df['als_히트'] = als_hit

In [30]:
als_inter = []

for val,als in zip(val_tag,als_tag):
    als_inter.append(len(set(val) - (set(val)-set(als))))
        
cal_tag_hit_df['als_히트_count'] = als_inter

In [31]:
print(f'이 예측 모델의 Hit Rate는 약 {np.round(sum(als_hit)/len(als_hit)*100,1)}% 입니다')

이 예측 모델의 Hit Rate는 약 79.6% 입니다


In [32]:
print(f'이 예측 모델의 정밀도는 약 {np.round(sum(als_inter)/10/len(als_inter)*100,2)}% 입니다')

이 예측 모델의 정밀도는 약 15.66% 입니다


## 예측된 태그 직접 보기

In [33]:
cal_tag_hit_df.head(40)

,예측용태그,검증용태그,als_예측결과,als_히트,als_히트_count
0,"[트로피컬하우스, 드라이브, 일렉]","[팝, 힐링, 운동, 트렌드, Pop, 2017, 기분전환]","[좋은노래, 팝송, 댄스, 세련된, 힙합, 띵곡, 취향저격, 클럽, 그루브, 소울,...",0,0
1,"[취향저격, 신나는, 여행]","[매장음악, 댄스, 여름, 드라이브, 일렉, 일렉트로니카, 기분전환]","[회상, 매장음악, 추억, 겨울, 드라이브, 운동, 감성, 사랑, 퇴근길, 설렘, ...",1,3
2,"[사랑, 감성, 우울]","[새벽, 인디, 발라드, 잔잔한, 이별, 알앤비, 기분전환]","[매장음악, 설렘, 여행, 겨울, 퇴근길, 새벽, 가을, 혼자, 봄, 기분전환]",1,2
3,"[드라이브, 라운지, 기분전환]","[세련된, 휴식, 펍, 매장, 그루브, 퇴근길, 새벽]","[매장음악, 여행, 비오는날, 여름, 감성, 매장, 카페, 새벽, 가을, 분위기, 밤]",1,2
4,"[잠, 추억, 잔잔한]","[매장음악, 일상, 추위, 매장, 카페, 자장가, 새벽]","[주말, 휴식, 비오는날, 힐링, 감성, 차분한, 카페, 배경음악, 노동요, 새벽,...",1,2
5,"[파워워킹, 퇴근길, 출근길]","[맥주, 드라이브, 비트, 흥겨운, 그루브, 분위기, 알앤비]","[매장음악, 설렘, 여행, 겨울, 감성, 산책, 사랑, 카페, 잔잔한, 새벽, 혼자...",0,0
6,"[Lofi, 밤, 저녁]","[캐럴, 재즈힙합, 눈, 오후, 연말, 자장가, 잠들기전]","[새벽감성, 매장음악, 센치, 여름밤, 퇴근길, 비, 우울, 새벽, 달달, 혼자, 설렘]",0,0
7,"[그냥, 위로, 그루브]","[팝송, 팝, 비오는날, 감성, Pop, 우울한, 기분전환]","[좋은노래, 혼자, 저녁, 슬픔, 힙합, 드라이브, 사랑, 퇴근길, 소울, 우울, ...",0,0
8,"[휴식, Chill, 잔잔한]","[팝, 감성, Pop, 새벽, 새벽에, 몽환한, 밤]","[매장음악, 여행, 힐링, 비오는날, 감성, 산책, 카페, 까페, 새벽, 밤, 기분전환]",1,3
9,"[여행, 드라이브, 힙합]","[힐링, 휴식, 오후, 그루브, 트렌디, 알앤비, 기분전환]","[매장음악, 회상, 추억, 운동, 감성, 사랑, 퇴근길, 새벽, 가을, 설렘, 기분전환]",1,1
